In [147]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df

import datetime
from datetime import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import normalize, StandardScaler

from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor  
from sklearn.tree import export_graphviz
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import get_scorer_names
from sklearn.metrics import r2_score, mean_squared_error # squared=False
from sklearn.metrics import explained_variance_score, mean_absolute_percentage_error

from sklearn.feature_selection import f_regression, SelectKBest, mutual_info_regression

import os


In [22]:
AIR_TEMP = 'air' # 1.Daily surface air temp. (K)
PRECIP_AMT = 'apcp' # 2.Daily accumulated total surface precip. (kg/m2)
CONV_PRECIP_AMT = 'acpcp' # 3.Daily accumulated conv. surface precip. (kg/m2)
TOTAL_CLOUD_COVER = 'tcdc' # 4.Daily forecast total cloud cover. (%)
UWND = 'uwnd' # 5.Daily u-wind eastward. (m/s)
DEW_POINT_TEMP = 'dpt' # 6.Daily dew point temp. (K)
ALBEDO = 'albedo' # 7.Daily surface albedo. (%)
POT_TEMP = 'pottmp' # 8.Daily potential surface temp. (K)
VWND = 'vwnd' # 9.Daily v-wind nothward. (m/s)
RH = 'rhum' # 10.Daily RH. (%)
WND = 'wnd' # 11. (u-wnd^2 + vwnd^2)^0.5 - net windspeed magnitude (m/s)

PLANT_CODE = 'plant_code'
LATITUDE = 'lat'
LONGITUDE = 'lon'
MONTH = 'month'
YEAR = 'year'

MAX = '_max'
MIN = '_min'
AVG = '_avg'

NAMEPLATE_CAPACITY = 'nameplate_capacity_mw'
ENERGY_SOURCE = 'energy_source_1'
GENERATION = 'net_gen'
PERFORMANCE_RATIO = 'performance_ratio'

MODEL1 = 'climate_cap_gen'
MODEL2 = 'climate_gen'
MODEL3 = 'cap_gen'

SCORING_METRICS = ['neg_root_mean_squared_error', 'r2', 'neg_mean_absolute_percentage_error']
KFOLDCV = 5


In [3]:
print(os.getcwd())
DATA_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Data/'
RESULTS_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Results/'
RESULTS_ALL_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Results/ALL/'

CODE_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code/'


/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code


In [4]:
raw_data_df = pd.read_excel(DATA_FOLDER + 'raw_master_df.xlsx')
print(raw_data_df.shape)
neg_cap_data_df = raw_data_df[raw_data_df[NAMEPLATE_CAPACITY] <= 0]
neg_gen_data_df = raw_data_df[raw_data_df[GENERATION] <= 0]
print(neg_cap_data_df.shape, neg_gen_data_df.shape)
raw_data_df = raw_data_df[(
    raw_data_df[NAMEPLATE_CAPACITY] > 0) & (
    raw_data_df[GENERATION] > 0)]
print(raw_data_df.shape)
raw_data_df.head()

(306576, 36)
(1373, 36) (10678, 36)
(295725, 36)


,plant_code,year,month,acpcp_min,acpcp_max,acpcp_avg,air_min,air_max,air_avg,albedo_min,...,tcdc_avg,uwnd_min,uwnd_max,uwnd_avg,vwnd_min,vwnd_max,vwnd_avg,energy_source_1,nameplate_capacity_mw,net_gen
0,2,2011,1,0.0,17.757812,0.611873,269.307343,288.326172,277.948092,10.0,...,45.600806,0.147941,4.579306,1.831262,0.268705,4.993980,2.192999,WAT,45.0,11312.437
1,2,2011,2,0.0,8.242188,0.526839,273.286682,294.365631,283.260946,10.0,...,35.709821,0.178692,4.874641,1.748523,0.199846,7.026369,2.905407,WAT,45.0,9090.908
2,2,2011,3,0.0,23.429688,2.082684,281.186005,294.162506,288.654109,10.0,...,38.161290,0.017701,5.251187,1.819689,0.267129,6.170527,3.009324,WAT,45.0,22446.993
3,2,2011,4,0.0,40.454475,5.850930,283.376892,297.169800,291.931956,10.0,...,25.220833,0.065307,6.336984,1.506685,0.146513,10.212384,3.824454,WAT,45.0,20390.027
4,2,2011,5,0.0,18.414062,0.967468,285.724548,300.778992,294.387196,20.0,...,25.858871,0.231988,3.841439,1.378504,0.202084,5.578331,2.654214,WAT,45.0,7243.627


#### 1. get wnd^2 = (uwnd^2 + vwnd^2) -> wnd_min, wnd_max, wnd_avg

In [5]:
for ent in [MIN, MAX, AVG]:
    raw_data_df[WND + ent] = (raw_data_df[VWND + ent]**2 + raw_data_df[UWND + ent]**2)**0.5

print(raw_data_df.shape)
raw_data_df.head()

(295725, 39)


,plant_code,year,month,acpcp_min,acpcp_max,acpcp_avg,air_min,air_max,air_avg,albedo_min,...,uwnd_avg,vwnd_min,vwnd_max,vwnd_avg,energy_source_1,nameplate_capacity_mw,net_gen,wnd_min,wnd_max,wnd_avg
0,2,2011,1,0.0,17.757812,0.611873,269.307343,288.326172,277.948092,10.0,...,1.831262,0.268705,4.993980,2.192999,WAT,45.0,11312.437,0.306739,6.775683,2.857055
1,2,2011,2,0.0,8.242188,0.526839,273.286682,294.365631,283.260946,10.0,...,1.748523,0.199846,7.026369,2.905407,WAT,45.0,9090.908,0.268085,8.551724,3.390977
2,2,2011,3,0.0,23.429688,2.082684,281.186005,294.162506,288.654109,10.0,...,1.819689,0.267129,6.170527,3.009324,WAT,45.0,22446.993,0.267715,8.102491,3.516717
3,2,2011,4,0.0,40.454475,5.850930,283.376892,297.169800,291.931956,10.0,...,1.506685,0.146513,10.212384,3.824454,WAT,45.0,20390.027,0.160409,12.018742,4.110541
4,2,2011,5,0.0,18.414062,0.967468,285.724548,300.778992,294.387196,20.0,...,1.378504,0.202084,5.578331,2.654214,WAT,45.0,7243.627,0.307663,6.773067,2.990840


In [6]:
# raw_data_df[PERFORMANCE_RATIO] = raw_data_df[GENERATION] / raw_data_df[NAMEPLATE_CAPACITY]

#### 2. EDA

In [7]:
climate_var_list = [col for col in raw_data_df.columns if (
    '_min' in col or '_max' in col or '_avg' in col)]
# climate_var_list = [col for col in raw_data_df.columns if ('_avg' in col)]

print(climate_var_list, len(climate_var_list))

['acpcp_min', 'acpcp_max', 'acpcp_avg', 'air_min', 'air_max', 'air_avg', 'albedo_min', 'albedo_max', 'albedo_avg', 'apcp_min', 'apcp_max', 'apcp_avg', 'dpt_min', 'dpt_max', 'dpt_avg', 'pottmp_min', 'pottmp_max', 'pottmp_avg', 'rhum_min', 'rhum_max', 'rhum_avg', 'tcdc_min', 'tcdc_max', 'tcdc_avg', 'uwnd_min', 'uwnd_max', 'uwnd_avg', 'vwnd_min', 'vwnd_max', 'vwnd_avg', 'wnd_min', 'wnd_max', 'wnd_avg'] 33


In [8]:
capacity_df = raw_data_df[[NAMEPLATE_CAPACITY]]
climate_var_df = raw_data_df[climate_var_list]
target_df = raw_data_df[[GENERATION]]


features_df_1 = raw_data_df[climate_var_list + [NAMEPLATE_CAPACITY]]
features_df_2 = climate_var_df
features_df_3 = capacity_df

features_dict = {MODEL1: features_df_1, MODEL2: features_df_2, MODEL3: features_df_3}

In [9]:
print(features_df_1.shape)
features_df_1.describe()

(295725, 34)


,acpcp_min,acpcp_max,acpcp_avg,air_min,air_max,air_avg,albedo_min,albedo_max,albedo_avg,apcp_min,...,uwnd_min,uwnd_max,uwnd_avg,vwnd_min,vwnd_max,vwnd_avg,wnd_min,wnd_max,wnd_avg,nameplate_capacity_mw
count,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,...,295725.000000,295725.000000,295725.000000,2.957250e+05,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000
mean,0.000065,8.234079,0.804532,277.997710,291.784255,285.244155,18.857347,26.603943,21.768310,0.001020,...,0.167584,5.886948,2.279185,1.475509e-01,5.935173,2.304280,0.261823,8.550538,3.338039,64.260974
std,0.003470,10.534889,1.158923,12.591953,10.265369,11.115728,7.172165,13.875418,9.308961,0.013124,...,0.297090,2.314745,0.950324,1.913365e-01,2.282701,0.863991,0.325848,2.709151,1.005724,222.568749
min,0.000000,0.000000,0.000000,237.365723,258.740601,248.320709,6.000000,6.000000,6.000000,0.000000,...,0.000000,0.694769,0.269055,9.536743e-07,0.681028,0.254705,0.000079,1.285487,0.461250,0.100000
25%,0.000000,0.843750,0.048360,269.149445,284.398010,276.775185,20.000000,20.000000,20.000000,0.000000,...,0.034151,4.168901,1.641356,3.473365e-02,4.294105,1.716508,0.096477,6.506147,2.644134,3.000000
50%,0.000000,3.843750,0.278610,278.539612,292.975769,285.765195,20.000000,20.000000,20.000000,0.000000,...,0.087298,5.547136,2.147007,8.847237e-02,5.709156,2.226470,0.179689,8.316617,3.235550,12.000000
75%,0.000000,11.906250,1.116160,287.622009,299.439331,293.860651,20.000000,30.000000,20.937500,0.000000,...,0.190601,7.278618,2.737432,1.897551e-01,7.343430,2.813254,0.312654,10.263824,3.898059,60.000000
max,1.070312,59.984375,15.284635,311.927917,317.756409,314.096387,80.000000,80.000000,80.000000,1.890625,...,6.767403,19.794159,9.832802,3.750830e+00,23.277878,8.997823,6.767408,25.025381,10.593750,6809.000000


In [10]:
print(features_df_2.shape)
features_df_2.describe()

(295725, 33)


,acpcp_min,acpcp_max,acpcp_avg,air_min,air_max,air_avg,albedo_min,albedo_max,albedo_avg,apcp_min,...,tcdc_avg,uwnd_min,uwnd_max,uwnd_avg,vwnd_min,vwnd_max,vwnd_avg,wnd_min,wnd_max,wnd_avg
count,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000,...,295725.000000,295725.000000,295725.000000,295725.000000,2.957250e+05,295725.000000,295725.000000,295725.000000,295725.000000,295725.000000
mean,0.000065,8.234079,0.804532,277.997710,291.784255,285.244155,18.857347,26.603943,21.768310,0.001020,...,39.522159,0.167584,5.886948,2.279185,1.475509e-01,5.935173,2.304280,0.261823,8.550538,3.338039
std,0.003470,10.534889,1.158923,12.591953,10.265369,11.115728,7.172165,13.875418,9.308961,0.013124,...,17.235960,0.297090,2.314745,0.950324,1.913365e-01,2.282701,0.863991,0.325848,2.709151,1.005724
min,0.000000,0.000000,0.000000,237.365723,258.740601,248.320709,6.000000,6.000000,6.000000,0.000000,...,0.185484,0.000000,0.694769,0.269055,9.536743e-07,0.681028,0.254705,0.000079,1.285487,0.461250
25%,0.000000,0.843750,0.048360,269.149445,284.398010,276.775185,20.000000,20.000000,20.000000,0.000000,...,26.153226,0.034151,4.168901,1.641356,3.473365e-02,4.294105,1.716508,0.096477,6.506147,2.644134
50%,0.000000,3.843750,0.278610,278.539612,292.975769,285.765195,20.000000,20.000000,20.000000,0.000000,...,39.287500,0.087298,5.547136,2.147007,8.847237e-02,5.709156,2.226470,0.179689,8.316617,3.235550
75%,0.000000,11.906250,1.116160,287.622009,299.439331,293.860651,20.000000,30.000000,20.937500,0.000000,...,52.112903,0.190601,7.278618,2.737432,1.897551e-01,7.343430,2.813254,0.312654,10.263824,3.898059
max,1.070312,59.984375,15.284635,311.927917,317.756409,314.096387,80.000000,80.000000,80.000000,1.890625,...,96.282258,6.767403,19.794159,9.832802,3.750830e+00,23.277878,8.997823,6.767408,25.025381,10.593750


In [11]:
print(features_df_3.shape)
features_df_3.describe()

(295725, 1)


,nameplate_capacity_mw
count,295725.000000
mean,64.260974
std,222.568749
min,0.100000
25%,3.000000
50%,12.000000
75%,60.000000
max,6809.000000


In [12]:
print(target_df.shape)
target_df.describe()

(295725, 1)


,net_gen
count,2.957250e+05
mean,1.652475e+04
std,6.915891e+04
min,5.000000e-03
25%,6.297830e+02
50%,2.840271e+03
75%,1.339200e+04
max,3.793524e+06


In [13]:
corr_df_dict = {MODEL1: features_df_1.corr(),
                MODEL2: features_df_2.corr()}
corr_df_dict[MODEL1].to_excel(RESULTS_ALL_FOLDER + MODEL1 + '_corr.xlsx')
corr_df_dict[MODEL2].to_excel(RESULTS_ALL_FOLDER + MODEL2 + '_corr.xlsx')

corr_df = raw_data_df[climate_var_list + [NAMEPLATE_CAPACITY, GENERATION]].corr()
corr_df.to_excel(RESULTS_ALL_FOLDER + 'all_corr.xlsx')

#### 3. Train-Test-Split (75/25)

In [14]:
# train_, test_ = train_test_split(raw_data_df, test_size=0.25)
train_, test_ = train_test_split(raw_data_df, test_size=0.25, stratify=raw_data_df[MONTH])
train_.shape, test_.shape

((221793, 39), (73932, 39))

In [15]:
train_features_dict = {}
test_features_dict = {}

train_target_df = train_[target_df.columns]
train_targets_ = train_target_df[GENERATION]
test_target_df = test_[target_df.columns]
test_targets_ = test_target_df[GENERATION]
print(train_target_df.shape, train_targets_.shape, test_target_df.shape, test_targets_.shape)

for key in features_dict:
    train_features_dict[key] = train_[features_dict[key].columns]
    test_features_dict[key] = test_[features_dict[key].columns]
    print(key, train_features_dict[key].shape, test_features_dict[key].shape)


(221793, 1) (221793,) (73932, 1) (73932,)
climate_cap_gen (221793, 34) (73932, 34)
climate_gen (221793, 33) (73932, 33)
cap_gen (221793, 1) (73932, 1)


#### 4. Data standardize -> (x - mu)/sigma -> N(0, 1)

In [16]:
train_features_scaled_dict = {}
test_features_scaled_dict = {}

for key in train_features_dict:
    print(key, '#################')
    scaler = StandardScaler().fit(train_features_dict[key])
    print(scaler.mean_)
    print(scaler.scale_)
    train_features_scaled_dict[key] = scaler.transform(train_features_dict[key])
    print(train_features_scaled_dict[key].mean(axis=0))
    print(train_features_scaled_dict[key].std(axis=0))
    test_features_scaled_dict[key] = scaler.transform(test_features_dict[key])
    print(test_features_scaled_dict[key].mean(axis=0))
    print(test_features_scaled_dict[key].std(axis=0))

    train_features_scaled_dict[key] = df(train_features_scaled_dict[key],
                                        columns=train_features_dict[key].columns)
    print(train_features_scaled_dict[key].shape)
    test_features_scaled_dict[key] = df(test_features_scaled_dict[key],
                                        columns=test_features_dict[key].columns)
    print(test_features_scaled_dict[key].shape)


climate_cap_gen #################
[7.07868441e-05 8.23339247e+00 8.03634334e-01 2.77997309e+02
 2.91788576e+02 2.85249125e+02 1.88596105e+01 2.66052216e+01
 2.17703177e+01 1.00682739e-03 1.93793870e+01 2.23468226e+00
 2.67996540e+02 2.84264030e+02 2.76804752e+02 2.82855530e+02
 2.97440746e+02 2.90515933e+02 4.52118694e+01 8.36643422e+01
 6.50670154e+01 3.28588594e+00 8.80748429e+01 3.95096883e+01
 1.67872840e-01 5.88799051e+00 2.27950311e+00 1.47261363e-01
 5.93138262e+00 2.30258435e+00 2.61829565e-01 8.54851973e+00
 3.33719941e+00 6.42981893e+01]
[3.87213010e-03 1.05243909e+01 1.15626437e+00 1.25955766e+01
 1.02643472e+01 1.11157958e+01 7.16788870e+00 1.38785109e+01
 9.30580873e+00 1.27754528e-02 1.58603942e+01 2.07580429e+00
 1.06620024e+01 7.83976474e+00 8.78961278e+00 1.36967864e+01
 1.10282677e+01 1.21710433e+01 1.89521995e+01 1.47759909e+01
 1.80195088e+01 5.99186747e+00 1.51532877e+01 1.72315199e+01
 2.97677588e-01 2.31296874e+00 9.49067499e-01 1.90786653e-01
 2.28148344e+00 8.6

In [18]:
# scaler = StandardScaler().fit(train_target_df)
# print(scaler.mean_)
# print(scaler.scale_)
# train_target_df_scaled = scaler.transform(train_target_df)
# print(train_target_df_scaled.mean(axis=0))
# print(train_target_df_scaled.std(axis=0))
# train_target_df_scaled = df(train_target_df_scaled, columns=[GENERATION])
# print(train_target_df_scaled.shape)

# test_target_df_scaled = scaler.transform(test_target_df)
# print(test_target_df_scaled.mean(axis=0))
# print(test_target_df_scaled.std(axis=0))
# test_target_df_scaled = df(test_target_df_scaled, columns=[GENERATION])
# print(test_target_df_scaled.shape)


In [17]:
train_target_df_scaled = train_target_df
test_target_df_scaled = test_target_df


#### 5. Models

In [23]:
# 1. Linear model - OLS Regression

ols_model_ = {}
ols_train_r2 = {}
ols_test_r2 = {}
ols_kfoldcv = {}

for key in train_features_scaled_dict:
    ols_ = LinearRegression()
    ols_.fit(train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    ols_model_[key] = ols_
    ols_train_r2[key] = ols_.score(
        train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    print(ols_train_r2[key])
    ols_test_r2[key] = ols_.score(
        test_features_scaled_dict[key], test_target_df_scaled[GENERATION])
    print(ols_test_r2[key])
    
    ols_coeff = ols_.coef_
    print(ols_coeff)
    ols_intercept = ols_.intercept_
    print(ols_intercept)
    ######################## kfold cv (k=5)
    ols_scores = cross_validate(
        ols_, train_features_scaled_dict[key], train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(ols_scores.keys()))
    ols_kfoldcv[key] = ols_scores
    print(ols_scores['train_r2'], np.mean(ols_scores['train_r2']))
    print(ols_scores['test_r2'], np.mean(ols_scores['test_r2']))
    print(ols_scores['train_neg_root_mean_squared_error'])
    print(ols_scores['test_neg_root_mean_squared_error'])
    print('##############', key)


0.7696604272012257
0.7767239416564209
[ 4.77314162e+01  2.85211050e+02 -6.34997138e+02  2.45595231e+03
 -1.06756569e+04  7.75376138e+03 -7.20359689e+01 -1.74773379e+02
 -9.90237240e+02 -1.37909336e+02 -9.05278623e+02  5.85917447e+02
  1.83393706e+03  1.24147629e+03 -2.47263843e+03 -2.95094418e+03
  9.60091954e+03 -7.91413665e+03  4.67180681e+02 -1.47467542e+03
  1.54900566e+03  3.93041168e+02 -6.87643237e+02  1.69179214e+03
  1.17621226e+03 -1.09104965e+03 -4.16583407e+02  6.32141114e+02
 -1.97899911e+03 -1.10816867e+03 -1.59935822e+03  1.49525037e+03
  1.66429145e+03  6.11795951e+04]
16570.085810440403
['fit_time', 'score_time', 'test_neg_mean_absolute_percentage_error', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_mean_absolute_percentage_error', 'train_neg_root_mean_squared_error', 'train_r2']
[0.76772255 0.77394384 0.76822297 0.76707058 0.77143085] 0.7696781579272269
[0.77704931 0.75137936 0.7757875  0.77890129 0.76209898] 0.7690432903533043
[-33488.99099834 -33381.961

In [24]:
# train_features_scaled_dict[MODEL1][f_rf_features]
# 1. Linear model - OLS Regression

ols_model_new = {}
ols_train_r2_new = {}
ols_test_r2_new = {}
ols_kfoldcv_new = {}

for key in train_features_scaled_dict:
    train_data = train_features_scaled_dict[key]
    test_data = test_features_scaled_dict[key]
    if key == MODEL1:
        train_data = train_data[f_rf_features + [NAMEPLATE_CAPACITY]]
        test_data = test_data[f_rf_features + [NAMEPLATE_CAPACITY]]
    elif key == MODEL2:
        train_data = train_data[f_rf_features]
        test_data = test_data[f_rf_features]
    ols_ = LinearRegression()
    ols_.fit(train_data, train_target_df_scaled[GENERATION])
    ols_model_new[key] = ols_
    ols_train_r2_new[key] = ols_.score(
        train_data, train_target_df_scaled[GENERATION])
    print(ols_train_r2_new[key])
    ols_test_r2_new[key] = ols_.score(
        test_data, test_target_df_scaled[GENERATION])
    print(ols_test_r2_new[key])
    
    ols_coeff = ols_.coef_
    print(ols_coeff)
    ols_intercept = ols_.intercept_
    print(ols_intercept)
    ######################## kfold cv (k=5)
    ols_scores = cross_validate(
        ols_, train_data, train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(ols_scores.keys()))
    ols_kfoldcv_new[key] = ols_scores
    print(ols_scores['train_r2'], np.mean(ols_scores['train_r2']))
    print(ols_scores['test_r2'], np.mean(ols_scores['test_r2']))
    print(ols_scores['train_neg_root_mean_squared_error'])
    print(ols_scores['test_neg_root_mean_squared_error'])
    print('##############', key)


NameError: name 'f_rf_features' is not defined

In [41]:
# 2. Linear model - Bayesian Regression

br_model_ = {}
br_train_r2 = {}
br_test_r2 = {}
br_kfoldcv = {}

for key in train_features_scaled_dict:
    br_ = BayesianRidge()
    br_.fit(train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    br_model_[key] = br_
    br_train_r2[key] = br_.score(
        train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    print(br_train_r2[key])
    br_test_r2[key] = br_.score(
        test_features_scaled_dict[key], test_target_df_scaled[GENERATION])
    print(br_test_r2[key])
    
    br_coeff = br_.coef_
    print(br_coeff)
    br_intercept = br_.intercept_
    print(br_intercept)
    ######################## kfold cv (k=5)
    br_scores = cross_validate(
        br_, train_features_scaled_dict[key], train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(br_scores.keys()))
    br_kfoldcv[key] = br_scores
    print(br_scores['train_r2'], np.mean(br_scores['train_r2']))
    print(br_scores['test_r2'], np.mean(br_scores['test_r2']))
    print(br_scores['train_neg_root_mean_squared_error'])
    print(br_scores['test_neg_root_mean_squared_error'])
    print('##############', key)


0.7696601243438125
0.7767276726884589
[ 4.79281413e+01  2.87201294e+02 -6.33390542e+02  2.91155626e+03
 -9.67945235e+03  6.25702731e+03 -6.98369588e+01 -1.69933658e+02
 -9.92595455e+02 -1.38233423e+02 -9.03176417e+02  5.79961582e+02
  1.82722411e+03  1.22842430e+03 -2.45002627e+03 -3.42416288e+03
  8.57512695e+03 -6.36370734e+03  4.71610858e+02 -1.47698865e+03
  1.54413028e+03  3.93092611e+02 -6.88367162e+02  1.68906004e+03
  1.17121420e+03 -1.07200426e+03 -4.24258829e+02  6.29293679e+02
 -1.96178293e+03 -1.11365233e+03 -1.59313026e+03  1.46560748e+03
  1.67611686e+03  6.11775500e+04]
16570.08581044039
['fit_time', 'score_time', 'test_neg_mean_absolute_percentage_error', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_mean_absolute_percentage_error', 'train_neg_root_mean_squared_error', 'train_r2']
[0.76772219 0.77394349 0.76822245 0.76707007 0.77143037] 0.7696777137518765
[0.77704646 0.75137707 0.77578886 0.77890133 0.76210372] 0.7690434877769257
[-33489.01735967 -33381.9870

In [42]:
# train_features_scaled_dict[MODEL1][f_rf_features]
# 2. Linear model - Bayesian Regression

br_model_new = {}
br_train_r2_new = {}
br_test_r2_new = {}
br_kfoldcv_new = {}

for key in train_features_scaled_dict:
    train_data = train_features_scaled_dict[key]
    test_data = test_features_scaled_dict[key]
    if key == MODEL1:
        train_data = train_data[f_rf_features + [NAMEPLATE_CAPACITY]]
        test_data = test_data[f_rf_features + [NAMEPLATE_CAPACITY]]
    elif key == MODEL2:
        train_data = train_data[f_rf_features]
        test_data = test_data[f_rf_features]

    br_ = BayesianRidge()
    br_.fit(train_data, train_target_df_scaled[GENERATION])
    br_model_new[key] = br_
    br_train_r2_new[key] = br_.score(
        train_data, train_target_df_scaled[GENERATION])
    print(br_train_r2_new[key])
    br_test_r2_new[key] = br_.score(
        test_data, test_target_df_scaled[GENERATION])
    print(br_test_r2_new[key])
    
    br_coeff = br_.coef_
    print(br_coeff)
    br_intercept = br_.intercept_
    print(br_intercept)
    ######################## kfold cv (k=5)
    br_scores = cross_validate(
        br_, train_data, train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(br_scores.keys()))
    br_kfoldcv_new[key] = br_scores
    print(br_scores['train_r2'], np.mean(br_scores['train_r2']))
    print(br_scores['test_r2'], np.mean(br_scores['test_r2']))
    print(br_scores['train_neg_root_mean_squared_error'])
    print(br_scores['test_neg_root_mean_squared_error'])
    print('##############', key)


NameError: name 'f_rf_features' is not defined

In [43]:
# 3. CART Decision Tree

cart_model_ = {}
cart_train_r2 = {}
cart_test_r2 = {}
cart_feature_importance = {}
cart_params = {}
cart_depth = {}
cart_max_features = {}
cart_feature_names = {}
cart_kfoldcv = {}

for key in train_features_scaled_dict:
    cart_ = DecisionTreeRegressor(random_state=2)
    cart_.fit(train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    cart_model_[key] = cart_
    cart_train_r2[key] = cart_.score(
        train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    print(cart_train_r2[key])
    cart_test_r2[key] = cart_.score(
        test_features_scaled_dict[key], test_target_df_scaled[GENERATION])
    print(cart_test_r2[key])
    ###############################
    cart_feature_importance[key] = cart_.feature_importances_
    cart_max_features[key] = cart_.max_features_
    cart_feature_names[key] = cart_.feature_names_in_
    cart_params[key] = cart_.get_params()
    cart_depth[key] = cart_.get_depth()
#     export_graphviz(cart_, out_file=key+'_cart_.dot',
#                     feature_names=train_features_scaled_dict[key])
    ######################## kfold cv (k=5)
    cart_scores = cross_validate(
        cart_, train_features_scaled_dict[key], train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(cart_scores.keys()))
    cart_kfoldcv[key] = cart_scores
    print(cart_scores['train_r2'], np.mean(cart_scores['train_r2']))
    print(cart_scores['test_r2'], np.mean(cart_scores['test_r2']))
    print(cart_scores['train_neg_root_mean_squared_error'])
    print(cart_scores['test_neg_root_mean_squared_error'])
    print('##############', key)


0.9997196770377201
0.8522486846258688
['fit_time', 'score_time', 'test_neg_mean_absolute_percentage_error', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_mean_absolute_percentage_error', 'train_neg_root_mean_squared_error', 'train_r2']
[0.99975061 0.99978273 0.99982025 0.99976714 0.9997651 ] 0.9997771650346617
[0.86752233 0.85741597 0.87889775 0.85333904 0.84862244] 0.8611595066878032
[-1097.34033178 -1034.91481431  -944.32135561 -1053.56437268
 -1068.68211644]
[-25825.08177766 -25692.2133626  -23353.77511513 -27828.33817614
 -27234.62778866]
############## climate_cap_gen
0.8823525971405312
-0.4864941720815481
['fit_time', 'score_time', 'test_neg_mean_absolute_percentage_error', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_mean_absolute_percentage_error', 'train_neg_root_mean_squared_error', 'train_r2']
[0.89475508 0.89206439 0.90081637 0.89001224 0.8939834 ] 0.8943262960521858
[-0.4276389  -0.39716472 -0.56626925 -0.51518972 -0.58944473] -0.4991414610592013
[

In [44]:
# train_features_scaled_dict[MODEL1][f_rf_features]
# 3. CART Decision Tree

cart_model_new = {}
cart_train_r2_new = {}
cart_test_r2_new = {}
cart_feature_importance_new = {}
cart_params_new = {}
cart_depth_new = {}
cart_max_features_new = {}
cart_feature_names_new = {}
cart_kfoldcv_new = {}

for key in train_features_scaled_dict:
    train_data = train_features_scaled_dict[key]
    test_data = test_features_scaled_dict[key]
    if key == MODEL1:
        train_data = train_data[f_rf_mi_common_features + [NAMEPLATE_CAPACITY]]
        test_data = test_data[f_rf_mi_common_features + [NAMEPLATE_CAPACITY]]
    elif key == MODEL2:
        train_data = train_data[f_rf_mi_common_features]
        test_data = test_data[f_rf_mi_common_features]

    cart_ = DecisionTreeRegressor(random_state=2)
    cart_.fit(train_data, train_target_df_scaled[GENERATION])
    cart_model_new[key] = cart_
    cart_train_r2_new[key] = cart_.score(
        train_data, train_target_df_scaled[GENERATION])
    print(cart_train_r2_new[key])
    cart_test_r2_new[key] = cart_.score(
        test_data, test_target_df_scaled[GENERATION])
    print(cart_test_r2_new[key])
    ###############################
    cart_feature_importance_new[key] = cart_.feature_importances_
    cart_max_features_new[key] = cart_.max_features_
    cart_feature_names_new[key] = cart_.feature_names_in_
    cart_params_new[key] = cart_.get_params()
    cart_depth_new[key] = cart_.get_depth()
#     export_graphviz(cart_, out_file=key+'_cart_.dot',
#                     feature_names=train_features_scaled_dict[key])
    ######################## kfold cv (k=5)
    cart_scores = cross_validate(
        cart_, train_data, train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(cart_scores.keys()))
    cart_kfoldcv_new[key] = cart_scores
    print(cart_scores['train_r2'], np.mean(cart_scores['train_r2']))
    print(cart_scores['test_r2'], np.mean(cart_scores['test_r2']))
    print(cart_scores['train_neg_root_mean_squared_error'])
    print(cart_scores['test_neg_root_mean_squared_error'])
    print('##############', key)


NameError: name 'f_rf_mi_common_features' is not defined

In [45]:
# 4. Ensemble - Gradient Boosting Tree

gbm_model_ = {}
gbm_train_r2 = {}
gbm_test_r2 = {}
gbm_feature_importance = {}
gbm_params = {}

gbm_train_score = {}
gbm_validation_score = {}

gbm_feature_names = {}
gbm_kfoldcv = {}

for key in train_features_scaled_dict:
    gbm_ = HistGradientBoostingRegressor(random_state = 2)
    gbm_.fit(train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    gbm_model_[key] = gbm_
    gbm_train_r2[key] = gbm_.score(
        train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    print(gbm_train_r2[key])
    gbm_test_r2[key] = gbm_.score(
        test_features_scaled_dict[key], test_target_df_scaled[GENERATION])
    print(gbm_test_r2[key])
    ###############################
#     gbm_feature_importance[key] = gbm_.feature_importances_
    gbm_feature_names[key] = gbm_.feature_names_in_
    gbm_params[key] = gbm_.get_params()
    gbm_train_score[key] = gbm_.train_score_
    gbm_validation_score[key] = gbm_.validation_score_
    ######################## kfold cv (k=5)
    gbm_scores = cross_validate(
        gbm_, train_features_scaled_dict[key], train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(gbm_scores.keys()))
    gbm_kfoldcv[key] = gbm_scores
    print(gbm_scores['train_r2'], np.mean(gbm_scores['train_r2']))
    print(gbm_scores['test_r2'], np.mean(gbm_scores['test_r2']))
    print(gbm_scores['train_neg_root_mean_squared_error'])
    print(gbm_scores['test_neg_root_mean_squared_error'])
    print('##############', key)


0.9209935317669075
0.8387283895578106
['fit_time', 'score_time', 'test_neg_mean_absolute_percentage_error', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_mean_absolute_percentage_error', 'train_neg_root_mean_squared_error', 'train_r2']
[0.94391271 0.94669002 0.94197578 0.94174537 0.94641104] 0.944146983355747
[0.85532444 0.8436428  0.86662637 0.83675381 0.81399881] 0.8432692482046299
[-16456.24611818 -16210.93008915 -16966.4025108  -16664.03918908
 -16141.48475062]
[-26987.82910941 -26904.50644165 -24508.45685317 -29359.69418026
 -30188.9877572 ]
############## climate_cap_gen
0.3834323414129426
0.16684358898640628
['fit_time', 'score_time', 'test_neg_mean_absolute_percentage_error', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_mean_absolute_percentage_error', 'train_neg_root_mean_squared_error', 'train_r2']
[0.38700355 0.38531766 0.39809953 0.40407045 0.4038299 ] 0.39566421850464
[0.18903707 0.20597926 0.23561637 0.18897198 0.16241756] 0.19640444786749697
[-54

In [46]:
# train_features_scaled_dict[MODEL1][f_rf_features]
# 4. Ensemble - Gradient Boosting Tree

gbm_model_new = {}
gbm_train_r2_new = {}
gbm_test_r2_new = {}
gbm_feature_importance_new = {}
gbm_params_new = {}

gbm_train_score_new = {}
gbm_validation_score_new = {}

gbm_feature_names_new = {}
gbm_kfoldcv_new = {}

for key in train_features_scaled_dict:
    train_data = train_features_scaled_dict[key]
    test_data = test_features_scaled_dict[key]
    if key == MODEL1:
        train_data = train_data[f_rf_mi_union_features + [NAMEPLATE_CAPACITY]]
        test_data = test_data[f_rf_mi_union_features + [NAMEPLATE_CAPACITY]]
    elif key == MODEL2:
        train_data = train_data[f_rf_mi_union_features]
        test_data = test_data[f_rf_mi_union_features]

    gbm_ = HistGradientBoostingRegressor(random_state = 2)
    gbm_.fit(train_data, train_target_df_scaled[GENERATION])
    gbm_model_new[key] = gbm_
    gbm_train_r2_new[key] = gbm_.score(
        train_data, train_target_df_scaled[GENERATION])
    print(gbm_train_r2_new[key])
    gbm_test_r2_new[key] = gbm_.score(
        test_data, test_target_df_scaled[GENERATION])
    print(gbm_test_r2_new[key])
    ###############################
#     gbm_feature_importance[key] = gbm_.feature_importances_
    gbm_feature_names_new[key] = gbm_.feature_names_in_
    gbm_params_new[key] = gbm_.get_params()
    gbm_train_score_new[key] = gbm_.train_score_
    gbm_validation_score_new[key] = gbm_.validation_score_
    ######################## kfold cv (k=5)
    gbm_scores = cross_validate(
        gbm_, train_data, train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(gbm_scores.keys()))
    gbm_kfoldcv_new[key] = gbm_scores
    print(gbm_scores['train_r2'], np.mean(gbm_scores['train_r2']))
    print(gbm_scores['test_r2'], np.mean(gbm_scores['test_r2']))
    print(gbm_scores['train_neg_root_mean_squared_error'])
    print(gbm_scores['test_neg_root_mean_squared_error'])
    print('##############', key)


NameError: name 'f_rf_mi_union_features' is not defined

In [190]:
# 5. Ensemble - RandomForest

rf_model_ = {}
rf_train_r2 = {}
rf_test_r2 = {}
rf_feature_importance = {}
rf_params = {}

# rf_train_score = {}
# rf_validation_score = {}

rf_feature_names = {}
rf_kfoldcv = {}

for key in train_features_scaled_dict:
    rf_ = RandomForestRegressor(random_state = 2)
    rf_.fit(train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    rf_model_[key] = rf_
    rf_train_r2[key] = rf_.score(
        train_features_scaled_dict[key], train_target_df_scaled[GENERATION])
    print(rf_train_r2[key])
    rf_test_r2[key] = rf_.score(
        test_features_scaled_dict[key], test_target_df_scaled[GENERATION])
    print(rf_test_r2[key])
    ###############################
    rf_feature_importance[key] = rf_.feature_importances_
    rf_feature_names[key] = rf_.feature_names_in_
    rf_params[key] = rf_.get_params()
#     rf_train_score[key] = rf_.train_score_
#     rf_validation_score[key] = rf_.validation_score_
    ######################## kfold cv (k=5)
    rf_scores = cross_validate(
        rf_, train_features_scaled_dict[key], train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(rf_scores.keys()))
    rf_kfoldcv[key] = rf_scores
    print(rf_scores['train_r2'], np.mean(rf_scores['train_r2']))
    print(rf_scores['test_r2'], np.mean(rf_scores['test_r2']))
    print(rf_scores['train_neg_root_mean_squared_error'])
    print(rf_scores['test_neg_root_mean_squared_error'])
    print('##############', key)



0.9910412041778667
0.9393863855320428
['fit_time', 'score_time', 'test_neg_mean_absolute_percentage_error', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_mean_absolute_percentage_error', 'train_neg_root_mean_squared_error', 'train_r2']
[0.99080136 0.99121114 0.99098315 0.99131051 0.99136676] 0.9911345848008251
[0.94333216 0.92561203 0.93918986 0.93233486 0.92389863] 0.9328735095891849
[-6664.38744028 -6582.18186805 -6688.25031378 -6435.94310673
 -6478.77388226]
[-16890.35172308 -18557.41309735 -16548.89399155 -18902.20673529
 -19310.20646668]
############## climate_cap_gen
0.796997014444812
0.14501192376292393
['fit_time', 'score_time', 'test_neg_mean_absolute_percentage_error', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_mean_absolute_percentage_error', 'train_neg_root_mean_squared_error', 'train_r2']
[0.8065126  0.80297271 0.80990068 0.80279428 0.80707615] 0.8058512842754265
[0.13794771 0.15496545 0.19195463 0.13250626 0.12061714] 0.1475982371317476
[-30565.

In [192]:
# 5. Ensemble - RandomForest

rf_model_new = {}
rf_train_r2_new = {}
rf_test_r2_new = {}
rf_feature_importance_new = {}
rf_params_new = {}

# rf_train_score = {}
# rf_validation_score = {}

rf_feature_names_new = {}
rf_kfoldcv_new = {}

for key in train_features_scaled_dict:
    train_data = train_features_scaled_dict[key]
    test_data = test_features_scaled_dict[key]
    if key == MODEL1:
        train_data = train_data[f_rf_mi_union_features + [NAMEPLATE_CAPACITY]]
        test_data = test_data[f_rf_mi_union_features + [NAMEPLATE_CAPACITY]]
    elif key == MODEL2:
        train_data = train_data[f_rf_mi_union_features]
        test_data = test_data[f_rf_mi_union_features]

    rf_ = RandomForestRegressor(random_state = 2)
    rf_.fit(train_data, train_target_df_scaled[GENERATION])
    rf_model_new[key] = rf_
    rf_train_r2_new[key] = rf_.score(
        train_data, train_target_df_scaled[GENERATION])
    print(rf_train_r2_new[key])
    rf_test_r2_new[key] = rf_.score(
        test_data, test_target_df_scaled[GENERATION])
    print(rf_test_r2_new[key])
    ###############################
    rf_feature_importance_new[key] = rf_.feature_importances_
    rf_feature_names_new[key] = rf_.feature_names_in_
    rf_params_new[key] = rf_.get_params()
#     rf_train_score[key] = rf_.train_score_
#     rf_validation_score[key] = rf_.validation_score_
    ######################## kfold cv (k=5)
    rf_scores = cross_validate(
        rf_, train_data, train_target_df_scaled[GENERATION],
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(rf_scores.keys()))
    rf_kfoldcv_new[key] = rf_scores
    print(rf_scores['train_r2'], np.mean(rf_scores['train_r2']))
    print(rf_scores['test_r2'], np.mean(rf_scores['test_r2']))
    print(rf_scores['train_neg_root_mean_squared_error'])
    print(rf_scores['test_neg_root_mean_squared_error'])
    print('##############', key)



NameError: name 'f_rf_mi_union_features' is not defined

##### https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

In [52]:
# 6. ANN - MLP
# learning_rate='adaptive'
# hidden_layer_sizes=(2,)
# max_iter=50

mlp_model_ = {}
mlp_train_r2 = {}
mlp_test_r2 = {}
mlp_nfeatures = {}
mlp_nlayers = {}
mlp_params = {}
mlp_kfoldcv = {}

for key in train_features_scaled_dict:
    mlp_ = MLPRegressor(random_state=2)
    mlp_.fit(train_features_scaled_dict[key].to_numpy(),
             train_target_df_scaled[GENERATION].to_numpy())
    mlp_model_[key] = mlp_
    mlp_train_r2[key] = mlp_.score(
        train_features_scaled_dict[key].to_numpy(),
        train_target_df_scaled[[GENERATION]].to_numpy())
    print(mlp_train_r2[key])
    mlp_test_r2[key] = mlp_.score(
        test_features_scaled_dict[key].to_numpy(),
        test_target_df_scaled[[GENERATION]].to_numpy())
    print(mlp_test_r2[key])
    
    mlp_nfeatures[key] = mlp_.n_features_in_
    mlp_nlayers[key] = mlp_.n_layers_
    mlp_params[key] = mlp_.get_params()
    ######################## kfold cv (k=5)
    mlp_scores = cross_validate(
        mlp_, train_features_scaled_dict[key].to_numpy(),
        train_target_df_scaled[[GENERATION]].to_numpy(),
        scoring=SCORING_METRICS, cv=KFOLDCV, return_train_score=True)
    print(sorted(mlp_scores.keys()))
    mlp_kfoldcv[key] = mlp_scores
    print(mlp_scores['train_r2'], np.mean(mlp_scores['train_r2']))
    print(mlp_scores['test_r2'], np.mean(mlp_scores['test_r2']))
    print(mlp_scores['train_neg_root_mean_squared_error'])
    print(mlp_scores['test_neg_root_mean_squared_error'])
    print('##############', key)


/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


0.8983297948175537
0.8988697736666734


/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/Use

['fit_time', 'score_time', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_root_mean_squared_error', 'train_r2']
[0.87917491 0.69187107 0.75148316 0.09628703 0.10390094] 0.5045434205932848
[0.88989189 0.69712071 0.76195008 0.11577014 0.10090607] 0.5131277763430726
[-0.34247471 -0.5533872  -0.50611908 -0.9534555  -0.94601874]
[-0.35069568 -0.5570605  -0.45690611 -0.92909708 -0.95062804]
############## climate_cap_gen


/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


...

['fit_time', 'score_time', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_root_mean_squared_error', 'train_r2']
[ 0.00663784 -0.13169854 -0.09926915 -0.0741763  -0.06203163] -0.07210755503589128
[ 0.00435339 -0.15660078 -0.11578262 -0.06907749 -0.05474567] -0.07837063430318784
[-0.98198253 -1.06054291 -1.06445381 -1.03949631 -1.02988908]
[-1.05456502 -1.08857739 -0.98919689 -1.02160521 -1.02963184]
############## climate_gen


/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


0.7648481858203574
0.797198507374308


/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vbc5136/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/Use

['fit_time', 'score_time', 'test_neg_root_mean_squared_error', 'test_r2', 'train_neg_root_mean_squared_error', 'train_r2']
[0.75580778 0.55906863 0.52934951 0.47150286 0.42786704] 0.5487191644793005
[0.77424744 0.56884801 0.5461572  0.48352689 0.40637671] 0.5558312492314417
[-0.48687319 -0.66198533 -0.69650471 -0.72913256 -0.75591021]
[-0.5021542  -0.66463433 -0.63087842 -0.71007196 -0.77243786]
############## cap_gen


#### 6. Feature Selection
##### Numerical feature selection - 1. Correlation, 2. Mutual information

In [214]:
def select_features(X_train, y_train, X_test, fxn):
    fs = SelectKBest(score_func=fxn, k=10)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs


#### make plots of k=10 best features selected

In [282]:
print(f_regr_features)

X_test.columns, np.argsort(fs.scores_)[-10:], fs.scores_[np.argsort(fs.scores_)[-10:]]

['acpcp_max', 'acpcp_avg', 'dpt_max', 'rhum_avg', 'wnd_max', 'apcp_avg', 'vwnd_avg', 'rhum_max', 'vwnd_max', 'apcp_max']


(Index(['acpcp_min', 'acpcp_max', 'acpcp_avg', 'air_min', 'air_max', 'air_avg',
        'albedo_min', 'albedo_max', 'albedo_avg', 'apcp_min', 'apcp_max',
        'apcp_avg', 'dpt_min', 'dpt_max', 'dpt_avg', 'pottmp_min', 'pottmp_max',
        'pottmp_avg', 'rhum_min', 'rhum_max', 'rhum_avg', 'tcdc_min',
        'tcdc_max', 'tcdc_avg', 'uwnd_min', 'uwnd_max', 'uwnd_avg', 'vwnd_min',
        'vwnd_max', 'vwnd_avg', 'wnd_min', 'wnd_max', 'wnd_avg'],
       dtype='object'),
 array([ 1,  2, 13, 20, 31, 11, 29, 19, 28, 10]),
 array([ 97.33426388, 103.39604269, 111.23919433, 139.42123418,
        177.0753617 , 184.08911652, 195.12443366, 202.6891264 ,
        261.49722244, 302.00448771]))

In [287]:
f_regr_features_df = df({'Feature': f_regr_features,
                         'F-statistic': fs.scores_[np.argsort(fs.scores_)[-10:]]})
f_regr_features_df

,Feature,F-statistic
0,acpcp_max,97.334264
1,acpcp_avg,103.396043
2,dpt_max,111.239194
3,rhum_avg,139.421234
4,wnd_max,177.075362
5,apcp_avg,184.089117
6,vwnd_avg,195.124434
7,rhum_max,202.689126
8,vwnd_max,261.497222
9,apcp_max,302.004488


In [327]:
plt.figure(figsize=(5,5))
# plt.grid()
ax = sns.barplot(f_regr_features_df, y='F-statistic', x='Feature',
                 linewidth=1.25, edgecolor=".3")
# ax.set_xlim([0, .1])
# ax.set_xlabel('F-statistic based Importance')
ax.set_title("F-statistic based Importance")
for p in ax.patches:
    ax.annotate("%.2f" % p.get_width(), (p.get_x() + p.get_width(), p.get_y()+.71),
                xytext=(4, 10), textcoords='offset points')
plt.savefig('plot_extra_f_regr_feature_importance.png',bbox_inches='tight')

In [328]:
X_train = train_features_scaled_dict[MODEL2]
y_train = train_target_df_scaled[GENERATION]
X_test = test_features_scaled_dict[MODEL2]

X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, f_regression)
# what are scores for the features
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

f_regr_features = [X_test.columns[ind] for ind in np.argsort(fs.scores_)[-10:]]

In [281]:
X_test.columns, np.argsort(fs.scores_)[-10:], fs.scores_[np.argsort(fs.scores_)[-10:]]

(Index(['acpcp_min', 'acpcp_max', 'acpcp_avg', 'air_min', 'air_max', 'air_avg',
        'albedo_min', 'albedo_max', 'albedo_avg', 'apcp_min', 'apcp_max',
        'apcp_avg', 'dpt_min', 'dpt_max', 'dpt_avg', 'pottmp_min', 'pottmp_max',
        'pottmp_avg', 'rhum_min', 'rhum_max', 'rhum_avg', 'tcdc_min',
        'tcdc_max', 'tcdc_avg', 'uwnd_min', 'uwnd_max', 'uwnd_avg', 'vwnd_min',
        'vwnd_max', 'vwnd_avg', 'wnd_min', 'wnd_max', 'wnd_avg'],
       dtype='object'),
 array([ 1,  2, 13, 20, 31, 11, 29, 19, 28, 10]),
 array([ 97.33426388, 103.39604269, 111.23919433, 139.42123418,
        177.0753617 , 184.08911652, 195.12443366, 202.6891264 ,
        261.49722244, 302.00448771]))

In [329]:
X_train = train_features_scaled_dict[MODEL2]
y_train = train_target_df_scaled[GENERATION]
X_test = test_features_scaled_dict[MODEL2]

X_train_fs, X_test_fs, mi = select_features(X_train, y_train, X_test, mutual_info_regression)
# what are scores for the features
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, mi.scores_[i]))
# plot the scores
plt.bar([i for i in range(len(mi.scores_))], mi.scores_)
plt.show()

mi_features = [X_test.columns[ind] for ind in np.argsort(mi.scores_)[-10:]]

In [270]:
rf_features_df = df(
    {'Feature': rf_feature_names[MODEL2],
     'Importance': rf_feature_importance[MODEL2]})
rf_features_df.head()

,Feature,Importance
0,acpcp_min,0.000037
1,acpcp_max,0.022235
2,acpcp_avg,0.021566
3,air_min,0.028640
4,air_max,0.022380


In [310]:
plt.figure(figsize=(8,10))
# plt.grid()
ax = sns.barplot(rf_features_df, x='Importance', y='Feature',
                 linewidth=1.5, edgecolor=".1")
ax.set_xlim([0, .1])
ax.set_xlabel('Mutual-Information based Importance')
ax.set_title("Feature Importance Plot")
for p in ax.patches:
    ax.annotate("%.3f" % p.get_width(), (p.get_x() + p.get_width(), p.get_y()+1),
                xytext=(3, 8), textcoords='offset points')
plt.savefig('plot_extra_rf_feature_importance.png',bbox_inches='tight')

In [311]:
fig, ax = plt.subplots()
plt.grid()
# plt.figsize(10,10)
rf_feature_imp_plot = plt.bar(rf_feature_names[MODEL2], rf_feature_importance[MODEL2])
plt.xticks(rotation=45)
plt.title('Feature Importance')
plt.savefig('all_rf_feature_imp.png')

rf_features = [X_test.columns[ind] for ind in np.argsort(rf_feature_importance[MODEL2])[-10:]]

In [278]:
rf_features = [X_test.columns[ind] for ind in np.argsort(rf_feature_importance[MODEL2])[-10:]]
print(f_regr_features)
print(mi_features)
print(rf_features)
print('#########')
f_rf_features = list(set(f_regr_features) & set(rf_features))
print(f_rf_features, len(f_rf_features))
print('#########')
f_mi_features = list(set(f_regr_features) & set(mi_features))
print(f_mi_features, len(f_mi_features))
print('#########')
mi_rf_features = list(set(mi_features) & set(mi_features))
print(mi_rf_features, len(mi_rf_features))
print('#########')
f_rf_mi_common_features = list(set(f_regr_features) & set(rf_features) & set(mi_features))
print(f_rf_mi_common_features, len(f_rf_mi_common_features))
print('#########')
f_rf_mi_union_features = list(set(f_regr_features) | set(rf_features) | set(mi_features))
print(f_rf_mi_union_features, len(f_rf_mi_union_features))


['acpcp_max', 'acpcp_avg', 'dpt_max', 'rhum_avg', 'wnd_max', 'apcp_avg', 'vwnd_avg', 'rhum_max', 'vwnd_max', 'apcp_max']
['pottmp_avg', 'dpt_max', 'air_avg', 'air_min', 'air_max', 'vwnd_avg', 'pottmp_max', 'rhum_min', 'albedo_max', 'rhum_avg']
['pottmp_avg', 'apcp_max', 'apcp_avg', 'dpt_min', 'rhum_min', 'rhum_max', 'vwnd_max', 'tcdc_avg', 'rhum_avg', 'vwnd_avg']
#########
['apcp_max', 'rhum_max', 'vwnd_max', 'rhum_avg', 'apcp_avg', 'vwnd_avg'] 6
#########
['dpt_max', 'vwnd_avg', 'rhum_avg'] 3
#########
['pottmp_max', 'pottmp_avg', 'dpt_max', 'air_min', 'rhum_avg', 'air_avg', 'rhum_min', 'vwnd_avg', 'air_max', 'albedo_max'] 10
#########
['vwnd_avg', 'rhum_avg'] 2
#########
['acpcp_avg', 'acpcp_max', 'dpt_max', 'air_max', 'air_min', 'rhum_min', 'vwnd_avg', 'albedo_max', 'pottmp_max', 'wnd_max', 'apcp_max', 'pottmp_avg', 'rhum_max', 'air_avg', 'vwnd_max', 'rhum_avg', 'dpt_min', 'apcp_avg', 'tcdc_avg'] 19


#### Results

In [179]:
# ols_train_r2_list = [ols_train_r2[MODEL3], ols_train_r2[MODEL1]]
# ols_test_r2_list = [ols_test_r2[MODEL3], ols_test_r2[MODEL1]]
# print(ols_train_r2_list, ols_test_r2_list)

# br_train_r2_list = [br_train_r2[MODEL3], br_train_r2[MODEL1]]
# br_test_r2_list = [br_test_r2[MODEL3], br_test_r2[MODEL1]]
# print(br_train_r2_list, br_test_r2_list)

# cart_train_r2_list = [cart_train_r2[MODEL3], cart_train_r2[MODEL1]]
# cart_test_r2_list = [cart_test_r2[MODEL3], cart_test_r2[MODEL1]]
# print(cart_train_r2_list, cart_test_r2_list)

# gbm_train_r2_list = [gbm_train_r2[MODEL3], gbm_train_r2[MODEL1]]
# gbm_test_r2_list = [gbm_test_r2[MODEL3], gbm_test_r2[MODEL1]]
# print(gbm_train_r2_list, gbm_test_r2_list)

# rf_train_r2_list = [rf_train_r2[MODEL3], rf_train_r2[MODEL1]]
# rf_test_r2_list = [rf_test_r2[MODEL3], rf_test_r2[MODEL1]]
# print(rf_train_r2_list, rf_test_r2_list)


In [178]:
# all_train_model3_r2_list = np.asarray(
#     [ols_train_r2[MODEL3], br_train_r2[MODEL3],
#      cart_train_r2[MODEL3], gbm_train_r2[MODEL3], rf_train_r2[MODEL3]])

# all_test_model3_r2_list = np.asarray(
#     [ols_test_r2[MODEL3], br_test_r2[MODEL3],
#      cart_test_r2[MODEL3], gbm_test_r2[MODEL3], rf_test_r2[MODEL3]])

# all_train_model1_r2_list = np.asarray(
#     [ols_train_r2[MODEL1], br_train_r2[MODEL1],
#      cart_train_r2[MODEL1], gbm_train_r2[MODEL1], rf_train_r2[MODEL1]])

# all_test_model1_r2_list = np.asarray(
#     [ols_test_r2[MODEL1], br_test_r2[MODEL1],
#      cart_test_r2[MODEL1], gbm_test_r2[MODEL1], rf_test_r2[MODEL1]])
# models = ['OLS', 'BR', 'CART', 'GBM', 'RF']

#### ~Plot 1: show - xaxis [(ols, br, cart, gbm, rf) -> MODEL1 train-test] yaxis [r2]~
#### ~Plot 2: show - xaxis [(ols, br, cart, gbm, rf) -> MODEL1 train-test] yaxis [rmse]~
#### ~Plot 3: show - xaxis [(ols, br, cart, gbm, rf) -> MODEL1 train-test] yaxis [nrmse]~

#### ~Plot 4: show - xaxis [(ols, br, cart, gbm, rf) -> MODEL1 kfold cv train-validate] yaxis [r2]~
#### ~Plot 5: show - xaxis [(ols, br, cart, gbm, rf) -> MODEL1 kfold cv train-validate] yaxis [rmse]~
#### ~Plot 6: show - xaxis [(ols, br, cart, gbm, rf) -> MODEL1 kfold cv train-validate] yaxis [nrmse]~


In [330]:
# plot 1
ols_train_test_r2 = [ols_test_r2[MODEL1], ols_train_r2[MODEL1]]
br_train_test_r2 = [br_test_r2[MODEL1], br_train_r2[MODEL1]]
cart_train_test_r2 = [cart_train_r2[MODEL1], cart_test_r2[MODEL1]]
gbm_train_test_r2 = [gbm_train_r2[MODEL1], gbm_test_r2[MODEL1]]
rf_train_test_r2 = [rf_train_r2[MODEL1], rf_test_r2[MODEL1]]
MODELS = ['OLS', 'BR', 'CART', 'GBM', 'RF']
x = ['Train (75%)', 'Test (25%)']

X_axis = np.arange(len(x)) 
width = 0.1
fig, ax = plt.subplots()

rects1 = ax.bar(X_axis, ols_train_test_r2, width, label = 'OLS')
rects2 = ax.bar(X_axis + width, br_train_test_r2, width, label = 'BR')
rects3 = ax.bar(X_axis + 2*width, cart_train_test_r2, width, label = 'CART')
rects4 = ax.bar(X_axis + 3*width, gbm_train_test_r2, width, label = 'GBM')
rects5 = ax.bar(X_axis + 4*width, rf_train_test_r2, width, label = 'RF')
ax.set_title('Comparing Train and Test ' '$R^2$' + ' across Generation Models')
ax.set_ylabel('$R^2$')
ax.text(0.5, 0.95,
        r'$R^2 = 1 - \frac{\Sigma_i (y_i - y_{pred})^2}{\Sigma_i (y_i - y_{mean})^2}$',
        fontsize=12)
ax.set_xticks(X_axis + width / 2)
ax.set_xticklabels(x) 
# ax.set_ylim([0, ])
ax.legend() #loc='lower right'

def autolabel(rects, factor):
    """
    Attach a text label above each bar displaying its height
    """
    for ind, rect in enumerate(rects):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2,
                factor*height,'%.2f' % height,ha='center', va='bottom',
                fontdict=dict(fontsize=9))

rects_list = [rects1, rects2, rects3, rects4, rects5]
for ind, rect in enumerate(rects_list):
    if ind/2 != 0:
        autolabel(rect, 1.01)
    else:
        autolabel(rect, 1.04)
# plt.grid()
plt.savefig('plot1_train_test_r2.png')
plt.show()


In [202]:
ols_test_rmse = mean_squared_error(
    test_target_df_scaled,
    ols_model_[MODEL1].predict(test_features_scaled_dict[MODEL1]),
    squared=False)
ols_train_rmse = mean_squared_error(
    train_target_df_scaled,
    ols_model_[MODEL1].predict(train_features_scaled_dict[MODEL1]),
    squared=False)

br_test_rmse = mean_squared_error(
    test_target_df_scaled,
    br_model_[MODEL1].predict(test_features_scaled_dict[MODEL1]),
    squared=False)
br_train_rmse = mean_squared_error(
    train_target_df_scaled,
    br_model_[MODEL1].predict(train_features_scaled_dict[MODEL1]),
    squared=False)
cart_test_rmse = mean_squared_error(
    test_target_df_scaled,
    cart_model_[MODEL1].predict(test_features_scaled_dict[MODEL1]),
    squared=False)
cart_train_rmse = mean_squared_error(
    train_target_df_scaled,
    cart_model_[MODEL1].predict(train_features_scaled_dict[MODEL1]),
    squared=False)
gbm_test_rmse = mean_squared_error(
    test_target_df_scaled,
    gbm_model_[MODEL1].predict(test_features_scaled_dict[MODEL1]),
    squared=False)
gbm_train_rmse = mean_squared_error(
    train_target_df_scaled,
    gbm_model_[MODEL1].predict(train_features_scaled_dict[MODEL1]),
    squared=False)
rf_test_rmse = mean_squared_error(
    test_target_df_scaled,
    rf_model_[MODEL1].predict(test_features_scaled_dict[MODEL1]),
    squared=False)
rf_train_rmse = mean_squared_error(
    train_target_df_scaled,
    rf_model_[MODEL1].predict(train_features_scaled_dict[MODEL1]),
    squared=False)

print(ols_test_rmse, ols_train_rmse)
print(br_test_rmse, br_train_rmse)
print(cart_test_rmse, cart_train_rmse)
print(gbm_test_rmse, gbm_train_rmse)
print(rf_test_rmse, rf_train_rmse)
TEST_DATA_STD = list(np.std(test_target_df_scaled))[0]
print(TEST_DATA_STD)

31778.976981289692 33491.00954830383
31778.71146048673 33491.03156579102
25851.426458907394 1168.3509000721008
27008.333987810594 19614.411606072215
16557.849288431786 6604.938526930663
67254.11065610986


In [331]:
# plot 3
ols_train_test_rmse = np.asarray([ols_test_rmse, ols_train_rmse]) / TEST_DATA_STD
br_train_test_rmse = np.asarray([br_test_rmse, br_train_rmse]) / TEST_DATA_STD
cart_train_test_rmse = np.asarray([cart_train_rmse, cart_test_rmse]) / TEST_DATA_STD
gbm_train_test_rmse = np.asarray([gbm_train_rmse, gbm_test_rmse]) / TEST_DATA_STD
rf_train_test_rmse = np.asarray([rf_train_rmse, rf_test_rmse]) / TEST_DATA_STD
MODELS = ['OLS', 'BR', 'CART', 'GBM', 'RF']
x = ['Train (75%)', 'Test (25%)']

X_axis = np.arange(len(x)) 
width = 0.1
fig, ax = plt.subplots()

rects1 = ax.bar(X_axis, ols_train_test_rmse, width, label = 'OLS')
rects2 = ax.bar(X_axis + width, br_train_test_rmse, width, label = 'BR')
rects3 = ax.bar(X_axis + 2*width, cart_train_test_rmse, width, label = 'CART')
rects4 = ax.bar(X_axis + 3*width, gbm_train_test_rmse, width, label = 'GBM')
rects5 = ax.bar(X_axis + 4*width, rf_train_test_rmse, width, label = 'RF')
ax.set_title('Train and Test ' 'NRMSE' + ' across Generation Models')
ax.set_ylabel('NRMSE')
ax.text(0.4, 0.35, r'$NRMSE = \frac{RMSE}{\sigma_{test}}$', fontsize=15)
ax.set_xticks(X_axis + width / 2)
ax.set_xticklabels(x) 
# ax.set_ylim([0, ])
ax.legend() #loc='lower right'

def autolabel(rects, factor):
    """
    Attach a text label above each bar displaying its height
    """
    for ind, rect in enumerate(rects):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2,
                factor*height,'%.2f' % height,ha='center', va='bottom',
                fontdict=dict(fontsize=9))

rects_list = [rects1, rects2, rects3, rects4, rects5]
for ind, rect in enumerate(rects_list):
    autolabel(rect, 1)
#     if ind/2 != 0:
#         autolabel(rect, 1.01)
#     else:
#         autolabel(rect, 1.04)
# plt.grid()
plt.savefig('plot3_train_test_nrmse.png')
plt.show()


In [332]:
# plot 2
ols_train_test_rmse = np.asarray([ols_test_rmse, ols_train_rmse])
br_train_test_rmse = np.asarray([br_test_rmse, br_train_rmse])
cart_train_test_rmse = np.asarray([cart_train_rmse, cart_test_rmse])
gbm_train_test_rmse = np.asarray([gbm_train_rmse, gbm_test_rmse])
rf_train_test_rmse = np.asarray([rf_train_rmse, rf_test_rmse])
MODELS = ['OLS', 'BR', 'CART', 'GBM', 'RF']
x = ['Train (75%)', 'Test (25%)']

X_axis = np.arange(len(x)) 
width = 0.1
fig, ax = plt.subplots()

rects1 = ax.bar(X_axis, ols_train_test_rmse, width, label = 'OLS')
rects2 = ax.bar(X_axis + width, br_train_test_rmse, width, label = 'BR')
rects3 = ax.bar(X_axis + 2*width, cart_train_test_rmse, width, label = 'CART')
rects4 = ax.bar(X_axis + 3*width, gbm_train_test_rmse, width, label = 'GBM')
rects5 = ax.bar(X_axis + 4*width, rf_train_test_rmse, width, label = 'RF')
ax.set_title('Train and Test ' 'RMSE' + ' across Generation Models')
ax.set_ylabel('RMSE')
ax.text(0.3, 30000, r'$RMSE = \sqrt{\frac{\Sigma_i (y_i - y_{pred})^2}{N}}$', fontsize=12)
ax.set_xticks(X_axis + width / 2)
ax.set_xticklabels(x) 
ax.set_ylim([0, 36000])
ax.legend() #loc='lower right'

def autolabel(rects, factor, orientation):
    """
    Attach a text label above each bar displaying its height
    """
    for ind, rect in enumerate(rects):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2,
                factor*height,'%d' % height,ha=orientation, va='bottom',
                fontdict=dict(fontsize=7))

rects_list = [rects1, rects2, rects3, rects4, rects5]
for ind, rect in enumerate(rects_list):
#     autolabel(rect, 1)
    if ind/2 != 0:
        autolabel(rect, 1.02, 'center')
    else:
        autolabel(rect, 1.01, 'right')
# plt.grid()
plt.savefig('plot2_train_test_rmse.png')
plt.show()


In [333]:
# plot 4a
test_r2_df = df({'Model': [], '$R^2$': []})
test_r2_df = pd.concat(
    [test_r2_df, df(
        {'model': ['OLS' for ent in ols_kfoldcv[MODEL1]['test_r2']],
         '$R^2$': ols_kfoldcv[MODEL1]['test_r2']})])
test_r2_df = pd.concat(
    [test_r2_df, df(
        {'model': ['BR' for ent in br_kfoldcv[MODEL1]['test_r2']],
         '$R^2$': br_kfoldcv[MODEL1]['test_r2']})])
test_r2_df = pd.concat(
    [test_r2_df, df(
        {'model': ['CART' for ent in cart_kfoldcv[MODEL1]['test_r2']],
         '$R^2$': cart_kfoldcv[MODEL1]['test_r2']})])
test_r2_df = pd.concat(
    [test_r2_df, df(
        {'model': ['GBM' for ent in gbm_kfoldcv[MODEL1]['test_r2']],
         '$R^2$': gbm_kfoldcv[MODEL1]['test_r2']})])
test_r2_df = pd.concat(
    [test_r2_df, df(
        {'model': ['RF' for ent in rf_kfoldcv[MODEL1]['test_r2']],
         '$R^2$': rf_kfoldcv[MODEL1]['test_r2']})])

# train_r2_df = pd.concat(
#     [train_r2_df, df(
#         {'model': ['MLP' for ent in mlp_kfoldcv[MODEL1]['train_r2']],
#          'train_r2': mlp_kfoldcv[MODEL1]['train_r2']})])
fig, ax = plt.subplots()
plt.grid()
ax.set_title('5-fold CV: Validation $R^2$ across Generation Models')

sns.boxplot(data=test_r2_df, x="model", y="$R^2$",
            showmeans=True, meanprops={'marker':'o',
                                       'markerfacecolor':'white',
                                       'markeredgecolor':'black'})
plt.savefig('plot4a_cv_validate_r2.png')

In [334]:
# plot 4b
train_r2_df = df({'Model': [], '$R^2$': []})
train_r2_df = pd.concat(
    [train_r2_df, df(
        {'Model': ['OLS' for ent in ols_kfoldcv[MODEL1]['train_r2']],
         '$R^2$': ols_kfoldcv[MODEL1]['train_r2']})])
train_r2_df = pd.concat(
    [train_r2_df, df(
        {'Model': ['BR' for ent in br_kfoldcv[MODEL1]['train_r2']],
         '$R^2$': br_kfoldcv[MODEL1]['train_r2']})])
train_r2_df = pd.concat(
    [train_r2_df, df(
        {'Model': ['CART' for ent in cart_kfoldcv[MODEL1]['train_r2']],
         '$R^2$': cart_kfoldcv[MODEL1]['train_r2']})])
train_r2_df = pd.concat(
    [train_r2_df, df(
        {'Model': ['GBM' for ent in gbm_kfoldcv[MODEL1]['train_r2']],
         '$R^2$': gbm_kfoldcv[MODEL1]['train_r2']})])
train_r2_df = pd.concat(
    [train_r2_df, df(
        {'Model': ['RF' for ent in rf_kfoldcv[MODEL1]['train_r2']],
         '$R^2$': rf_kfoldcv[MODEL1]['train_r2']})])

# train_r2_df = pd.concat(
#     [train_r2_df, df(
#         {'model': ['MLP' for ent in mlp_kfoldcv[MODEL1]['train_r2']],
#          'train_r2': mlp_kfoldcv[MODEL1]['train_r2']})])
fig, ax = plt.subplots()
plt.grid()
ax.set_title('5-fold CV: Train $R^2$ across Generation Models')

sns.boxplot(data=train_r2_df, x="Model", y="$R^2$",
            showmeans=True, meanprops={'marker':'o',
                                       'markerfacecolor':'white',
                                       'markeredgecolor':'black'})
plt.savefig('plot4b_cv_train_r2.png')

In [209]:
print(ols_kfoldcv[MODEL3]['test_neg_root_mean_squared_error']/ list(
    np.std(test_target_df_scaled))[0])
print(ols_kfoldcv[MODEL3]['test_neg_root_mean_squared_error'], np.mean(
    ols_kfoldcv[MODEL3]['test_neg_root_mean_squared_error']))
print(ols_kfoldcv[MODEL3]['train_neg_root_mean_squared_error']/ list(
    np.std(train_target_df_scaled))[0])
print(ols_kfoldcv[MODEL3]['train_neg_root_mean_squared_error'], np.mean(
    ols_kfoldcv[MODEL3]['train_neg_root_mean_squared_error']))


[-0.50027932 -0.50671269 -0.47485826 -0.51012076 -0.50949568]
[-33645.84088008 -34078.51143488 -31936.17015448 -34307.71790162
 -34265.67887099] -33646.783848407795
[-0.48203509 -0.48047143 -0.48801078 -0.47965186 -0.47991713]
[-33637.41500738 -33528.29968483 -34054.41140849 -33471.108605
 -33489.61920044] -33636.1707812297


In [335]:
# plot 5a
test_rmse_df = df({'Model': [], 'RMSE': []})
test_rmse_df = pd.concat(
    [test_rmse_df, df(
        {'Model': ['OLS' for ent in ols_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'RMSE': -1*ols_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']})])
test_rmse_df = pd.concat(
    [test_rmse_df, df(
        {'Model': ['BR' for ent in br_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'RMSE': -1*br_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']})])
test_rmse_df = pd.concat(
    [test_rmse_df, df(
        {'Model': ['CART' for ent in cart_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'RMSE': -1*cart_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']})])
test_rmse_df = pd.concat(
    [test_rmse_df, df(
        {'Model': ['GBM' for ent in gbm_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'RMSE': -1*gbm_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']})])
test_rmse_df = pd.concat(
    [test_rmse_df, df(
        {'Model': ['RF' for ent in rf_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'RMSE': -1*rf_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']})])

# train_r2_df = pd.concat(
#     [train_r2_df, df(
#         {'model': ['MLP' for ent in mlp_kfoldcv[MODEL1]['train_r2']],
#          'train_r2': mlp_kfoldcv[MODEL1]['train_r2']})])
fig, ax = plt.subplots()
plt.grid()
ax.set_title('5-fold CV: Validation RMSE across Generation Models')

sns.boxplot(data=test_rmse_df, x="Model", y="RMSE",
            showmeans=True, meanprops={'marker':'o',
                                       'markerfacecolor':'white',
                                       'markeredgecolor':'black'})
plt.savefig('plot5a_cv_validate_rmse.png')

In [336]:
# plot 5b
train_rmse_df = df({'Model': [], 'RMSE': []})
train_rmse_df = pd.concat(
    [train_rmse_df, df(
        {'Model': ['OLS' for ent in ols_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'RMSE': -1*ols_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']})])
train_rmse_df = pd.concat(
    [train_rmse_df, df(
        {'Model': ['BR' for ent in br_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'RMSE': -1*br_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']})])
train_rmse_df = pd.concat(
    [train_rmse_df, df(
        {'Model': ['CART' for ent in cart_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'RMSE': -1*cart_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']})])
train_rmse_df = pd.concat(
    [train_rmse_df, df(
        {'Model': ['GBM' for ent in gbm_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'RMSE': -1*gbm_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']})])
train_rmse_df = pd.concat(
    [train_rmse_df, df(
        {'Model': ['RF' for ent in rf_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'RMSE': -1*rf_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']})])

# train_r2_df = pd.concat(
#     [train_r2_df, df(
#         {'model': ['MLP' for ent in mlp_kfoldcv[MODEL1]['train_r2']],
#          'train_r2': mlp_kfoldcv[MODEL1]['train_r2']})])
fig, ax = plt.subplots()
plt.grid()
ax.set_title('5-fold CV: Train RMSE across Generation Models')

sns.boxplot(data=train_rmse_df, x="Model", y="RMSE",
            showmeans=True, meanprops={'marker':'o',
                                       'markerfacecolor':'white',
                                       'markeredgecolor':'black'})
plt.savefig('plot5b_cv_train_rmse.png')

In [337]:
# plot 6a
test_Nrmse_df = df({'Model': [], 'NRMSE': []})
test_Nrmse_df = pd.concat(
    [test_Nrmse_df, df(
        {'Model': ['OLS' for ent in ols_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'NRMSE': -1*ols_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']/TEST_DATA_STD})])
test_Nrmse_df = pd.concat(
    [test_Nrmse_df, df(
        {'Model': ['BR' for ent in br_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'NRMSE': -1*br_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']/TEST_DATA_STD})])
test_Nrmse_df = pd.concat(
    [test_Nrmse_df, df(
        {'Model': ['CART' for ent in cart_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'NRMSE': -1*cart_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']/TEST_DATA_STD})])
test_Nrmse_df = pd.concat(
    [test_Nrmse_df, df(
        {'Model': ['GBM' for ent in gbm_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'NRMSE': -1*gbm_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']/TEST_DATA_STD})])
test_Nrmse_df = pd.concat(
    [test_Nrmse_df, df(
        {'Model': ['RF' for ent in rf_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']],
         'NRMSE': -1*rf_kfoldcv[MODEL1]['test_neg_root_mean_squared_error']/TEST_DATA_STD})])

# train_r2_df = pd.concat(
#     [train_r2_df, df(
#         {'model': ['MLP' for ent in mlp_kfoldcv[MODEL1]['train_r2']],
#          'train_r2': mlp_kfoldcv[MODEL1]['train_r2']})])
fig, ax = plt.subplots()
plt.grid()
ax.set_title('5-fold CV: Validation NRMSE across Generation Models')

sns.boxplot(data=test_Nrmse_df, x="Model", y="NRMSE",
            showmeans=True, meanprops={'marker':'o',
                                       'markerfacecolor':'white',
                                       'markeredgecolor':'black'})
plt.savefig('plot6a_cv_validate_Nrmse.png')

In [338]:
# plot 6b
train_Nrmse_df = df({'Model': [], 'NRMSE': []})
train_Nrmse_df = pd.concat(
    [train_Nrmse_df, df(
        {'Model': ['OLS' for ent in ols_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'NRMSE': -1*ols_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']/TEST_DATA_STD})])
train_Nrmse_df = pd.concat(
    [train_Nrmse_df, df(
        {'Model': ['BR' for ent in br_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'NRMSE': -1*br_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']/TEST_DATA_STD})])
train_Nrmse_df = pd.concat(
    [train_Nrmse_df, df(
        {'Model': ['CART' for ent in cart_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'NRMSE': -1*cart_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']/TEST_DATA_STD})])
train_Nrmse_df = pd.concat(
    [train_Nrmse_df, df(
        {'Model': ['GBM' for ent in gbm_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'NRMSE': -1*gbm_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']/TEST_DATA_STD})])
train_Nrmse_df = pd.concat(
    [train_Nrmse_df, df(
        {'Model': ['RF' for ent in rf_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']],
         'NRMSE': -1*rf_kfoldcv[MODEL1]['train_neg_root_mean_squared_error']/TEST_DATA_STD})])

# train_r2_df = pd.concat(
#     [train_r2_df, df(
#         {'model': ['MLP' for ent in mlp_kfoldcv[MODEL1]['train_r2']],
#          'train_r2': mlp_kfoldcv[MODEL1]['train_r2']})])
fig, ax = plt.subplots()
plt.grid()
ax.set_title('5-fold CV: Train NRMSE across Generation Models')

sns.boxplot(data=train_Nrmse_df, x="Model", y="NRMSE",
            showmeans=True, meanprops={'marker':'o',
                                       'markerfacecolor':'white',
                                       'markeredgecolor':'black'})
plt.savefig('plot6b_cv_train_Nrmse.png')